In [6]:
import pandas as pd

# ucitavanje kompletnih podataka za manje fajlove
iupred_df = pd.read_csv('iupred_results_marburg.csv')
vsl2b_df = pd.read_csv('vsl2b_results_marburg.csv')

# preimenovanje kolona za manje fajlove
iupred_df = iupred_df.rename(columns={'Klasifikacija': 'IUPRED3_Klasifikacija'})
vsl2b_df = vsl2b_df.rename(columns={'Klasifikacija': 'VSL2B_Klasifikacija'})

# definisanje chunk-a za najveci fajl
chunk_size = 10000
isunstruct_df = pd.read_csv('/home/user/Desktop/IP2/IsUnstruct_2.02/isunstruct_results.csv', chunksize=chunk_size)

# za skladistenje spojenih podataka
merged_df = pd.DataFrame()

# iteracija kroz chunk-ove iz ISUNSTRUCT
for isunstruct_chunk in isunstruct_df:
    
    isunstruct_chunk = isunstruct_chunk.rename(columns={'Klasifikacija': 'ISUNSTRUCT_Klasifikacija'})

    # spajanje sa celim IUPRED3 DataFrame-om
    chunk_merged = pd.merge(
        isunstruct_chunk[['Ime fajla', 'Aminokiselina', 'Pozicija', 'ISUNSTRUCT_Klasifikacija']],
        iupred_df[['Ime fajla', 'Aminokiselina', 'Pozicija', 'IUPRED3_Klasifikacija']],
        on=['Ime fajla', 'Aminokiselina', 'Pozicija'],
        how='outer'
    )

    # spajanje sa celim VSL2B DataFrame-om
    final_chunk = pd.merge(
        chunk_merged,
        vsl2b_df[['Ime fajla', 'Aminokiselina', 'Pozicija', 'VSL2B_Klasifikacija']],
        on=['Ime fajla', 'Aminokiselina', 'Pozicija'],
        how='outer'
    )

    # dodavanje rezultata u finalni DataFrame
    merged_df = pd.concat([merged_df, final_chunk], ignore_index=True)

# uklanjanje duplikata
merged_df = merged_df.drop_duplicates()

# cuvanje spojenog DataFrame-a u CSV fajl
merged_df.to_csv('spojeni_rezultati.csv', index=False)

# prikaz prvih redova spojenog DataFrame-a za proveru
print("Spojeni DataFrame bez duplikata:")
print(merged_df.head())


Spojeni DataFrame bez duplikata:
       Ime fajla Aminokiselina  Pozicija ISUNSTRUCT_Klasifikacija  \
0  seq_001.fasta             M         1                neuređena   
1  seq_001.fasta             D         2                neuređena   
2  seq_001.fasta             L         3                neuređena   
3  seq_001.fasta             H         4                neuređena   
4  seq_001.fasta             S         5                neuređena   

  IUPRED3_Klasifikacija VSL2B_Klasifikacija  
0               uređena           neuređena  
1               uređena           neuređena  
2               uređena           neuređena  
3               uređena           neuređena  
4               uređena           neuređena  
